# ROS Developers Conference: Table segmentation with PCL  and TIAGo robot

<img src="img/pal-logo.png" width="400" />

The Point Cloud Library (PCL) is a standalone, large scale, open project for 2D/3D image and point cloud processing.
You can check more information about on the following link: http://pointclouds.org/

In the following Unit you are going to see different examples of Perception based on the PointCloud data that TIAGo camera provides using the PCL library. Basically, you are going to see the following:

* Table Segmentation
* Cylinder Detector
* Region Based Segmentation

But before actually getting to analyze each of the different examples, we'll need to set up TIAGo robot so it can have a better view of the objects on top of the table. For that, we're going to execute a motion that will raise TIAGo's torso and will lower its head. For doing this, you can follow the next steps.

## Launching the TIAGo simulation

First of all, let's launch the TIAGo simulation. For that, open a web shell on the **Tools** menu and then launch the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_gazebo pcl_launch.launch

Then, to see the simulation, go to **Tools->Gazebo**. A new window should appear, showing the TIAGo simulation.

<img src="img/pcl_lookdown.png" width="500" />

Then, we'll load an specific file with motion definitions.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosparam load `rospack find tiago_pcl_tutorial`/config/pcl_motions.yaml

Now, let's make sure that the command worked correctly and that the motions have been loaded.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosparam list

Among all the parameters, you should detect one that is called **/play_motion/motions/look_down/**.

<img src="img/pcl_params_list.png" width="600" />

Now, let's run axclient in order to send a goal to the **/play_motion** action.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun actionlib axclient.py /play_motion

and fill in the goal data like this:

<img src="img/pcl_axclient2.png" width="300" />

Finally, click on the **SEND GOAL** button, and you will see how TIAGo raises its torso and lowers its head.

<img src="img/pcl_lookdown.png" width="500" />

Great! Now TIAGo is ready to perfectly visualize the table and the objets on top of it.

## Table Segmentation

In this section you are going to see how to use the basic filters of PCL to achieve segmenting a table and all the objects on top of it.

Follow the next exercise in order to see a Pick & Place demo.

<p style="background:#EE9023;color:white;">Exercise 5.1</p>

a) Execute the following command in order to start the **segment_table** node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_pcl_tutorial segment_table.launch

Now, you should visualize in Rviz TIAGo in front of a colored point cloud representing the table, the objects on its top and a part of the floor.

<img src="img/pcl_cloud1.png" width="600" />

The **segment_table** node subscribes to the point cloud obtained by the RGBD camera of TIAGo's head and applies several filters in order to segment the table. The node first applies a passthrough filter in order to remove the floor points. Then, a downsampling is applied in order to speed up subsequent computations. Afterwards, the main plane, i.e. the table, is located using sample consensus segmentation. Finally, outliers are removed using statistical trimming of those points too far from its neighbours.

The node publishes two additional point clouds:

* **/segment_table/plane**
* **/segment_table/nonplane**

The first topic contains the portion of the original point cloud which has been considered taking part of the table.

<img src="img/pcl_cloud2.png" width="600" />

The second point cloud contains the points corresponding to object lying on top of the table.

<img src="img/pcl_cloud3.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 5.1</p>

Great! So now, you have already created a full map of the environment. So now what? Well, now it's time to save this map, so you can use it in the Pat Planning system!

## Cylinder Detector

In the following section you are going to see how to apply a sample consensus based segmentation algorithm from PCL in order to detect cylindrical objects in point clouds.

<p style="background:#EE9023;color:white;">Exercise 5.2</p>

a) Execute the following command in order to start the **segment_table** node with an extra argument.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_pcl_tutorial segment_table.launch show_rviz:=false

b) Execute now the following command in order to start the cylinder detector algorithm.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_pcl_tutorial cylinder_detector.launch

Rviz will show up showing three point clouds: the table point cloud, the non-table point cloud and a point cloud in yellow corresponding to the biggest cylinder fitted in the non-table point cloud. The cylindrical primitive best fitting the corresponding point cluster will be shown in cyan. Furthermore, the 3D pose of the reconstructed cylinder will be shown as a frame.

<img src="img/pcl_cylinder.png" width="600" />

<img src="img/pcl_cylinder2.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 5.2</p>

# How to launch the rosbag

If you want to avoid the simulation, you can test also using a rosbag.

I have included a rosbag with logged pointcloud information of the table. The rosbag is automatically included in the datasets_ws directory.

## First, close all the previous code
First you need to close all the previous programs.

Press Ctrl+C in all the openned shells.

## Second, let's launch the rosbag

Open another shell and launch a roscore:

In [ ]:
roscore

Open another shell and type the following:

The rosbag will start paused. You only need to hit space to start running the rosbag (but remember to have the focus of the shell when you press space).

In [ ]:
cd ~/datasets_ws/rosbag
rosbag play --pause -k --clock table_with_cylinders.bag

# Cylinder detector code in detail



The code of the cylinder_detector is available at: tiago_pcl_tutorial/src/nodes/cylinder_detector.cpp

The main part of the code is the PointCloud callback:

In [ ]:
void CylinderDetector::cloudCallback(const sensor_msgs::PointCloud2ConstPtr& cloud)
{
  
  if ( (cloud->width * cloud->height) == 0)
    return;

  pcl::PointCloud<pcl::PointXYZRGB>::Ptr pclCloud(new pcl::PointCloud<pcl::PointXYZRGB>);
  pcl::fromROSMsg(*cloud, *pclCloud);

  pcl::PointCloud<pcl::PointXYZRGB>::Ptr pclCylinderCloud(new pcl::PointCloud<pcl::PointXYZRGB>);
  pcl::ModelCoefficients::Ptr cylinderCoefficients(new pcl::ModelCoefficients);
  bool found = pal::cylinderSegmentation<pcl::PointXYZRGB>(pclCloud,
                                                           pclCylinderCloud,
                                                           10,
                                                           0.015, 0.08,
                                                           cylinderCoefficients);

  //filter outliers in the cylinder cloud
  pcl::PointCloud<pcl::PointXYZRGB>::Ptr pclFilteredCylinderCloud(new pcl::PointCloud<pcl::PointXYZRGB>);
  if ( pclCylinderCloud->empty() )
    pclFilteredCylinderCloud = pclCylinderCloud;
  else
    pal::statisticalOutlierRemoval<pcl::PointXYZRGB>(pclCylinderCloud, 25, 1.0,pclFilteredCylinderCloud);

  if ( found )
  {
    print(cylinderCoefficients, pclCylinderCloud->points.size());

    Eigen::Vector3d projectedCentroid, lineVector;
    getPointAndVector(pclFilteredCylinderCloud,
                      cylinderCoefficients,
                      projectedCentroid,
                      lineVector);

    double cylinderHeight = computeHeight(pclFilteredCylinderCloud,
                                          projectedCentroid,
                                          lineVector);

//    ROS_INFO_STREAM("The cylinder centroid is (" << centroid.head<3>().transpose() <<
//                    ") and projected to its axis is (" << projectedCentroid.transpose() << ")");

    Eigen::Matrix4d cylinderTransform;
    //create a frame given the cylinder parameters (point and vector)
    pal::pointAndLineTransform(lineVector,
                               projectedCentroid,
                               cylinderTransform);

    publish(pclFilteredCylinderCloud,
            cylinderCoefficients,
            cylinderTransform,
            cylinderHeight,
            cloud->header);
  }
}

The first function being called is the function pal::cylinderSegmentation, to retrieve the set of points that best fit the cylinder model, by using a RANSAC algorithm. The code of the function for more details is the following:

In [ ]:
/**
     * @brief cylinderSegmentation
     * @param inputCloud
     * @param cylinderCloud
     * @param neighbors number of neighbors considered to estimate the normals
     * @param cylinderCoefficients
     * @return
     */
    template <typename PointT>
    bool cylinderSegmentation(const typename pcl::PointCloud<PointT>::Ptr& inputCloud,
                              typename pcl::PointCloud<PointT>::Ptr& cylinderCloud,
                              int neighbors,
                              double minRadius,
                              double maxRadius,
                              pcl::ModelCoefficients::Ptr& cylinderCoefficients)
    {
      pcl::PointCloud<pcl::Normal>::Ptr normals(new pcl::PointCloud<pcl::Normal>);
      estimateNormals<PointT>(inputCloud,
                              neighbors,
                              normals);

      if ( normals->size() < 20 )
        return false;

      pcl::SACSegmentationFromNormals<PointT, pcl::Normal> seg;
      pcl::PointIndices::Ptr inliers(new pcl::PointIndices);

      seg.setOptimizeCoefficients(true);
      seg.setModelType(pcl::SACMODEL_CYLINDER);
      seg.setMethodType(pcl::SAC_RANSAC);
      seg.setNormalDistanceWeight (0.1);
      seg.setMaxIterations(10000);
      seg.setDistanceThreshold(0.05);
      seg.setRadiusLimits(minRadius, maxRadius);
      seg.setInputCloud(inputCloud);
      seg.setInputNormals(normals);
        
      // Best points fitting the cylinder model are retrieved
      seg.segment(*inliers, *cylinderCoefficients);

      if ( !inliers->indices.empty() )
      {
        //Extract inliers
        pcl::ExtractIndices<PointT> extract;
        extract.setInputCloud(inputCloud);
        extract.setIndices(inliers);
        extract.setNegative(false);
        extract.filter(*cylinderCloud);
        return true;
      }
      else
        return false;
    }


The inliers points that best fit a cylinder are than furtherly filtered using pal::statisticalOutlierRemoval, that uses pcl library function for filtering outliers from the cloud based on their distance from neighbors points:

In [ ]:
   template <typename PointT>
    void statisticalOutlierRemoval(const typename pcl::PointCloud<PointT>::Ptr& inputCloud,
                                   int numberOfPoints,
                                   double stdDevMult,
                                   typename pcl::PointCloud<PointT>::Ptr& outputCloud)
    {
      pcl::StatisticalOutlierRemoval<PointT> filter;
      filter.setInputCloud(inputCloud);
      filter.setMeanK(numberOfPoints);
      filter.setStddevMulThresh(stdDevMult);
      filter.filter(*outputCloud);
    }

In [ ]:
Finally the centroid of the cylinder is extracted

## Region Based Segmentation

In the following section you are going to see how to use a C++ node that subscribes to the point cloud of the RGBD camera of TIAGo's head and applies the PCL region growing tutorial (http://pointclouds.org/documentation/tutorials/region_growing_segmentation.php) to segment the scene in different clusters.

<p style="background:#EE9023;color:white;">Exercise 5.3</p>

a) Execute the following command in order to start the region bases segmentation demo.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_pcl_tutorial pcl_region.launch

Now, you should visualize in Rviz a different coloured point cloud for every different segmented region.

<img src="img/pcl_regions.png" width="600" />

<img src="img/pcl_regions2.png" width="600" />

Alongside with RViz, a rqt_reconfigure GUI has also appeared. Selecting the pcl_regions node, a set of parameters can be tunned.

<img src="img/pcl_regions_rqt.png" width="600" />

The parameters involve point cloud downsampling, in order to speed up the processing, and the region segmentation algorithm, which will have direct effect on the resulting number of regions and its morphology.

<p style="background:#EE9023;color:white;">End of Exercise 5.3</p>